# Hyperparameter Tuning

Determine the optimal combination of segments and periods for time series aggregation.

Author: Leander Kotzur

Import pandas and the relevant time series aggregation class

In [ ]:
%load_ext autoreload
%autoreload 2

import os

import pandas as pd

# Configure Plotly for sphinx/nbsphinx output
import plotly.io as pio

import tsam
from tsam import ClusterConfig
from tsam.tuning import find_pareto_front

pio.renderers.default = "notebook"

### Input data 

Read in time series from testdata.csv with pandas

In [ ]:
raw = pd.read_csv("testdata.csv", index_col=0)
raw = raw.rename(
    columns={
        "T": "Temperature [°C]",
        "Load": "Load [kW]",
        "Wind": "Wind [m/s]",
        "GHI": "Solar [W/m²]",
    }
)

Use tsam's built-in heatmap plotting for visual comparison of the time series

Plot an example series - in this case the temperature

In [ ]:
# Original data heatmaps
tsam.plot.heatmaps(raw, period_hours=24, title="Original Data")

### Tune a hierarchical aggregation with segments in combination with duration representation

Use the new `find_pareto_front()` function to explore the Pareto-optimal combinations of periods and segments.

In [ ]:
pareto_results = find_pareto_front(
    raw,
    period_hours=24,
    max_timesteps=8760,
    cluster=ClusterConfig(
        method="hierarchical",
        representation="duration",
    ),
    n_jobs=-1,
)

And determine the pareto optimal aggregation up to the full time series. This may take some time...

In [ ]:
# Show the last result in the Pareto front
last_result = pareto_results[-1]
print(
    f"Final configuration: {last_result.optimal_n_periods} periods, {last_result.optimal_n_segments} segments"
)
print(f"RMSE: {last_result.optimal_rmse:.4f}")

And show the results for the last aggregation

In [ ]:
# Reconstruct the original data from the last Pareto-optimal result
reconstructed = last_result.best_result.reconstruct()

In [ ]:
# Reconstructed data heatmaps
tsam.plot.heatmaps(
    reconstructed,
    reference_data=raw,
    period_hours=24,
    title="Reconstructed Data",
)

Save the resulting combination

In [ ]:
# Save the Pareto front results
pareto_df = pd.DataFrame(
    [
        {
            "segments": r.optimal_n_segments,
            "periods": r.optimal_n_periods,
            "rmse": r.optimal_rmse,
        }
        for r in pareto_results
    ]
)
pareto_df.to_csv(os.path.join("results", "paretoOptimalAggregation.csv"))